In [ ]:
%load_ext autoreload 
%autoreload 2 

In [ ]:
import random
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec # for subplots
import pandas as pd
import numpy as np
import seaborn as sns
import seaborn.objects as so
import tensorflow as tf
import json

from src.data_processing.pipelines.LSTMPipe import LSTMPipe

In [37]:
# DATA_PATH = '/projects/p31961/gaby_data/aggregated_data/data_pipeline_full_dataset/datasets/full_dataset.parquet.gzip'
# MODEL_PATH = "/projects/p31961/ENIGMA/results/experiments/dopamine_full_sequence_stacked_lstm_01/models/dopamine_full_sequence_stacked_lstm_01"

# local
DATA_PATH = '/Users/mds8301/iterm_data_storage/raw_data_raw_data.parquet.gzip'
MODEL_PATH = '/Users/mds8301/Devlopment/enigma/results/experiments/full_sequence_stacked_lstm_01/models/full_sequence_stacked_lstm_01'

In [ ]:
processor = LSTMPipe(DATA_PATH)
processor.read_raw_data()
processor.raw_data

In [ ]:
with open('/Users/mds8301/Devlopment/enigma/results/experiments/full_sequence_stacked_lstm_01/subjects.json', 'r') as f:
    subjects = json.load(f)
subjects

In [34]:
training_subjects = subjects['training']
dev_subjects = subjects['dev']
test_subjects = subjects['test']

training_query = ' or '.join([f"mouse_id=={subject}" for subject in training_subjects])
dev_query = ' or '.join([f"mouse_id=={subject}" for subject in dev_subjects])
test_query = ' or '.join([f"mouse_id=={subject}" for subject in test_subjects])

def split_by_subjects_query(subjects):
    query = ' or '.join([f"mouse_id=={subject}" for subject in subjects])
    x, y = processor.raw_data.query(query).drop(columns ="signal"), processor.raw_data.query(query)['signal']
    return x, y

processor.X_train, processor.y_train = split_by_subjects_query(training_subjects)
processor.X_dev, processor.y_dev = split_by_subjects_query(dev_subjects)
processor.X_test, processor.y_test = split_by_subjects_query(test_subjects)


In [35]:
processor.transorm_data()

In [38]:
model = tf.keras.models.load_model(MODEL_PATH)

model.compile(optimizer='adam', loss='mse', metrics=[
        'mae', 'mse', 'mape', 'cosine_similarity'])

/Users/mds8301/anaconda3/envs/enigma/lib/python3.11/site-packages/keras/src/layers/core/lambda_layer.py:327: UserWarning: src.models.experimental_dropout_StackedLSTM is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


In [39]:
model.summary()

Model: "stacked_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Lambda_1 (Lambda)           multiple                  0         
                                                                 
 LSTM_1 (LSTM)               multiple                  5472      
                                                                 
 Dropout_1 (Dropout)         multiple                  0         
                                                                 
 LSTM_2 (LSTM)               multiple                  10512     
                                                                 
 Dropout_2 (Dropout)         multiple                  0         
                                                                 
 Dense_output (Dense)        multiple                  37        
                                                                 
Total params: 16021 (62.58 KB)
Trainable params: 16021

In [40]:
predicted_signal = model.predict(processor.X_test)

37556/37556 [==============================] - 120s 3ms/step


In [41]:
x_test_raw, y_test_raw = split_by_subjects_query(test_subjects)
full_test_set = (x_test_raw
                 .assign(
                     true_signal=y_test_raw,
                     predicted_signal = predicted_signal
                     )
                 )


In [48]:
full_test_set.shape

(1201792, 13)

In [47]:
full_test_set.isnull().sum()

mouse_id                  0
day                       0
event                     0
sensor                    0
time                      0
trial                     0
action                  458
latency                   0
sex                       0
learning_phase            0
trial_count               0
true_signal             443
predicted_signal    1201792
dtype: int64

In [43]:
full_test_set

,mouse_id,day,event,sensor,time,trial,action,latency,sex,learning_phase,trial_count,true_signal,predicted_signal
0,12,5,cue,D2,-25.000000,0,escape,0.0,F,0,115,-0.155359,NaN
1,12,5,cue,D2,-24.901531,0,escape,0.0,F,0,115,-0.420553,NaN
2,12,5,cue,D2,-24.803064,0,escape,0.0,F,0,115,-1.592294,NaN
3,12,5,cue,D2,-24.704596,0,escape,0.0,F,0,115,-1.268734,NaN
4,12,5,cue,D2,-24.606127,0,escape,0.0,F,0,115,-0.210176,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5848197,5,4,escape,DA,19.606127,11,avoid,5.5,M,1,98,-0.010664,NaN
5848198,5,4,escape,DA,19.704596,11,avoid,5.5,M,1,98,0.472312,NaN
5848199,5,4,escape,DA,19.803064,11,avoid,5.5,M,1,98,0.005370,NaN
5848200,5,4,escape,DA,19.901531,11,avoid,5.5,M,1,98,-0.525126,NaN


In [62]:

processor_pipe = (LSTMPipe(DATA_PATH)
.read_raw_data(sort_by=['mouse_id','sensor','event', 'trial_count']))
processor.raw_data = processor_pipe.raw_data[::10000]
(processor_pipe.split_data(processed_data = False, 
            test_size=0.3,
            test_dev_size=0.5, 
            split_group = "mouse_id", 
            stratify_group = "sex", 
            target='signal', 
            save_subject_ids=False)
.transorm_data()
)

In [63]:
from src.models.experimental_dropout_StackedLSTM import StackedLSTM
def lr_schedular(epoch, lr):
        if epoch < 10:
            return lr
        else:
            return lr * tf.math.exp(-0.1)


model = StackedLSTM(
    sequence_length=processor.raw_data['time'].nunique(),
    num_features=processor.X_train.shape[1],
    lstm_units=processor.X_train.shape[1] * 2
)


model.compile(optimizer='adam', loss='mse', metrics=[
    'mae', 'mse', 'mape', 'cosine_similarity'])

# call backs


learning_rate_callback = tf.keras.callbacks.LearningRateScheduler(
    lr_schedular, verbose=1)

model.fit(processor.X_train,
            processor.y_train,
            epochs=5,
            validation_data=(processor.X_dev, processor.y_dev),
            callbacks=[learning_rate_callback]
            )


model.evaluate(processor.X_test, processor.y_test)






Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/5
  1725/116018 [..............................] - ETA: 25:03 - loss: nan - mae: nan - mse: nan - mape: nan - cosine_similarity: nan

KeyboardInterrupt: 

In [ ]:
sns.lineplot(data = avoid, x = 'time', y = 'signal', hue = 'signal_type')
# sns.lineplot(data = query, x = 'time', y = 'predicted_signal')

In [ ]:
facet = sns.FacetGrid(avoid, row = 'day', col = "learning_phase")
facet.map_dataframe(sns.lineplot, x = 'time', y = 'signal', hue = 'signal_type', hue_order = ["true_signal", "predicted_signal"])

In [ ]:
mouse_3 = avoid.query("mouse_id_3==1 & trial_count < 10")
mouse_3

In [ ]:
sns.lineplot(data = mouse_3.query("trial_count==9"), x = 'time', y = 'signal', hue = 'signal_type')

In [ ]:
evalulation = model.evaluate(X_test, y_test)
for name, value in zip(model.metrics_names, evalulation):
    print(f'{name}: {value}')

## Analysis on training set

In [ ]:
def query_testing_subjects(subjects, df):
    
    full_query = ''
    for mouse in subjects:
        query =f"{mouse} == 1"
        if full_query == '':
            full_query = query
        else:
            full_query += f" or {query}" 
        
    return df.query(full_query)

train_set = query_testing_subjects(subjects_by_category['training'], train_processor.data)
X_train, y_train = train_set.drop(columns = 'signal'), train_set['signal']

In [ ]:
predicted_signal = model.predict(X_train)

In [ ]:
full_train_set = (X_train
                 .assign(
                     true_signal=y_train,
                     predicted_signal = predicted_signal
                     )
                 )

In [ ]:
train_avoid = (full_train_set
         .query("action_avoid==1 & event_cue==1")
         .melt(id_vars = full_test_set.drop(columns = ['predicted_signal', 'true_signal']).columns, value_vars= ['predicted_signal', 'true_signal'], value_name = "signal", var_name = 'signal_type'))
train_avoid

In [ ]:
sns.lineplot(data = train_avoid, x = 'time', y = 'signal', hue = 'signal_type')

In [ ]:
facet = sns.FacetGrid(train_avoid, row = 'day', col = "learning_phase")
facet.map_dataframe(sns.lineplot, x = 'time', y = 'signal', hue = 'signal_type', hue_order = ["true_signal", "predicted_signal"])

In [ ]:
mouse_id_1_train  = train_avoid.query("mouse_id_1==1 & trial_count < 20")
sns.lineplot(data = mouse_id_1_train.query("trial_count==17"), x = 'time', y = 'signal', hue = 'signal_type')

# Summary
given the training performance data, I think this model is still overrfitting. It looks a little better, but validation loss is still increasing. I am going to expand on this and include a standard learning rate schedular in experiment 3
